In [1]:
import pandas as pd
import numpy as np
import shutil
import glob
import os

In [2]:
# Get names of strains sequenced by nanopore in the lab

# Get qc dataframe of all autocycler assemblies

# Load metrics
metrics = pd.read_csv('/n/grad_lab2/Lab/gonococcus/nanopore_genome_assemblies/autocycler/metrics.tsv', sep = '\t')
metrics['strain'] = metrics['name'].str.split('/', expand = True)[1]

# Basic QC

# Filter to genomes with at least 2 Mbp length and that are fully resolved
metrics_pass = metrics[(metrics['consensus_assembly_fully_resolved'])&
                       (metrics['consensus_assembly_bases']>2*10**6)]

wgs_ids = metrics_pass['strain'].values
nanopore=pd.DataFrame(wgs_ids, columns=['wgs_id'])

In [4]:
# Read in metadata
metadata = pd.read_csv('../../data/lab_strains_hybrid_genomes_metadata.csv', index_col = 0)
metadata.dropna(subset = ['pseudogenome_path'], inplace = True)

# Get metadata for strains sequenced by nanopore in the lab
merged = metadata.merge(nanopore, on = 'wgs_id', how = 'right', indicator = True)
complete_genomes = merged[merged['_merge'] == 'both'].drop('_merge', axis = 1)

# Check the "names" column for the strain names that were not found in the "wgs_id" column
no_match = merged[merged['_merge']=='right_only']['wgs_id'].values
for wgs_id in no_match:
    complete_genomes = pd.concat([complete_genomes, metadata[metadata['names'].astype('str').str.contains(wgs_id)]], ignore_index = True)

# Copy pseudogenome paths to run with Gubbins

paths = complete_genomes['pseudogenome_path'].values

for path in paths:
    shutil.copy(path, "../../data/nanopore_genome_pseudogenomes/")

In [5]:
# Correct format of filename to be {strain}_pseudogenome.fasta

os.rename("../../data/nanopore_genome_pseudogenomes/AUNG1065.fasta", "../../data/nanopore_genome_pseudogenomes/AUNG1065_pseudogenome.fasta")
os.rename("../../data/nanopore_genome_pseudogenomes/AUNG1251.fasta", "../../data/nanopore_genome_pseudogenomes/AUNG1251_pseudogenome.fasta")
os.rename("../../data/nanopore_genome_pseudogenomes/AUNG1291.fasta", "../../data/nanopore_genome_pseudogenomes/AUNG1291_pseudogenome.fasta")